In [0]:
# Dylan's MacbookPro #
# JP DILAN KALPA - 11634268 #
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


#Twitter credentials for the app
consumer_key = 'V2nHZg4htSMlYaJiVIHl5Xs9U'
consumer_secret = '35rja1sYYw7OTtMvfx6cZIU4gesccDWG2k5xKqK62qLtlUDWoZ'
access_key= '243015768-oKgiNxt5i7Hci8FTBTuNDthbBfHBlGr8PI5m3bnE'
access_secret = 'y5H9o2uIF1DYGapg3dydlpOaQRfzgdjsLqbpXRf2C1GtL'

#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#file location changed to "data/telemedicine_data_extraction/" for clearer path
telemedicine_tweets = "data/tweets_uribe.csv"
epilepsy_tweets = "data/tweets_ingrodolfo.csv"
heart_stroke_tweets = "data/tweets_duque.csv"

#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
       'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
       'user_mentions', 'place', 'place_coord_boundaries']

#set two date variables for date range
start_date = '2020-01-01'
end_date = '2020-01-31'

# Happy Emoticons
emoticons_happy = set([
   ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
   ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
   '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
   'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
   '<3'
   ])

# Sad Emoticons
emoticons_sad = set([
   ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
   ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
   ':c', ':{', '>:\\', ';('
   ])

#Emoji patterns
emoji_pattern = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticons
                          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                          u"\U0001F680-\U0001F6FF"  # transport & map symbols
                          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          u"\U00002702-\U000027B0"
                          u"\U000024C2-\U0001F251"
                          "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)


#mrhod clean_tweets()
def clean_tweets(tweet):
   stop_words = set(stopwords.words('english'))
   word_tokens = word_tokenize(tweet)

   #after tweepy preprocessing the colon left remain after removing mentions
   #or RT sign in the beginning of the tweet
   tweet = re.sub(r':', '', tweet)
   tweet = re.sub(r'‚Ä¶', '', tweet)
   #replace consecutive non-ASCII characters with a space
   tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)


   #remove emojis from tweet
   tweet = emoji_pattern.sub(r'', tweet)

   #filter using NLTK library append it to a string
   filtered_tweet = [w for w in word_tokens if not w in stop_words]
   filtered_tweet = []

   #looping through conditions
   for w in word_tokens:
       #check tokens against stop words , emoticons and punctuations
       if w not in stop_words and w not in emoticons and w not in string.punctuation:
           filtered_tweet.append(w)
   return ' '.join(filtered_tweet)
   #print(word_tokens)
   #print(filtered_sentence)

#method write_tweets()
def write_tweets(keyword, file):
   # If the file exists, then read the existing data from the CSV file.
   if os.path.exists(file):
       df = pd.read_csv(file, header=0)
   else:
       df = pd.DataFrame(columns=COLS)
   #page attribute in tweepy.cursor and iteration
   for page in tweepy.Cursor(api.search, q=keyword,
                             count=500, include_rts=False, since=start_date).pages(50):
       for status in page:
           new_entry = []
           status = status._json

           ## check whether the tweet is in english or skip to the next tweet
           if status['lang'] != 'en':
               continue

           #when run the code, below code replaces the retweet amount and
           #no of favorires that are changed since last download.
           if status['created_at'] in df['created_at'].values:
               i = df.loc[df['created_at'] == status['created_at']].index[0]
               if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                  status['retweet_count'] != df.at[i, 'retweet_count']:
                   df.at[i, 'favorite_count'] = status['favorite_count']
                   df.at[i, 'retweet_count'] = status['retweet_count']
               continue

   #tweepy preprocessing called for basic preprocessing
           clean_text = p.clean(status['text'])

           #call clean_tweet method for extra preprocessing
           filtered_tweet=clean_tweets(clean_text)

           #pass textBlob method for sentiment calculations
           blob = TextBlob(filtered_tweet)
           Sentiment = blob.sentiment

           #seperate polarity and subjectivity in to two variables
           polarity = Sentiment.polarity
           subjectivity = Sentiment.subjectivity

           #new entry append
           new_entry += [status['id'], status['created_at'],
                         status['source'], status['text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                         status['favorite_count'], status['retweet_count']]

           #to append original author of the tweet
           new_entry.append(status['user']['screen_name'])

           try:
               is_sensitive = status['possibly_sensitive']
           except KeyError:
               is_sensitive = None
           new_entry.append(is_sensitive)

           # hashtagas and mentiones are saved using comma separted
           hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
           new_entry.append(hashtags)
           mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
           new_entry.append(mentions)

       #get location of the tweet if possible
           try:
               location = status['user']['location']
           except TypeError:
               location = ''
           new_entry.append(location)

           try:
               coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
           except TypeError:
               coordinates = None
           new_entry.append(coordinates)

           single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
           df = df.append(single_tweet_df, ignore_index=True)
           csvFile = open(file, 'a' ,encoding='utf-8')
   df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

#declare keywords as a query for three categories
telemedicine_keywords = '@AlvaroUribeVel'
Epilepsy_keywords = 'ingrodolfohdez'
HeartDisease_keywords = 'IvanDuque'

#call main method passing keywords and file path
write_tweets(telemedicine_keywords,  telemedicine_tweets)
#write_tweets(Epilepsy_keywords, epilepsy_tweets)
#write_tweets(HeartDisease_keywords, heart_stroke_tweets)

# New Section

In [2]:
pip install tweet-preprocessor

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=cf0f3b86d45c3e3a86bd9ca49d55d33a9c76fa069c45a4205fdb5532982bd646
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
import tweepy
import csv
import pandas as pd
####input your credentials here
consumer_key =  'V2nHZg4htSMlYaJiVIHl5Xs9U'
consumer_secret = '35rja1sYYw7OTtMvfx6cZIU4gesccDWG2k5xKqK62qLtlUDWoZ'
access_token = '243015768-oKgiNxt5i7Hci8FTBTuNDthbBfHBlGr8PI5m3bnE'
access_token_secret = 'y5H9o2uIF1DYGapg3dydlpOaQRfzgdjsLqbpXRf2C1GtL'

'''
#Twitter credentials for the app
consumer_key = 'V2nHZg4htSMlYaJiVIHl5Xs9U'
consumer_secret = '35rja1sYYw7OTtMvfx6cZIU4gesccDWG2k5xKqK62qLtlUDWoZ'
access_key= '243015768-oKgiNxt5i7Hci8FTBTuNDthbBfHBlGr8PI5m3bnE'
access_secret = 'y5H9o2uIF1DYGapg3dydlpOaQRfzgdjsLqbpXRf2C1GtL'
'''



auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####United Airlines
# Open/Create a file to append data
csvFile = open('ua.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="ingrodolfohdez",count=100,
                           include_rts=False,
                           since="2019-01-01").items():
    print (tweet.created_at, tweet.text)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

2020-01-28 21:41:29 RT @ALDOMMATUK: @bettizaa @SergioGonzalezF @MabelLaraNews @gustavopetro @ClaudiaLopez @6AMCaracol Se lea dijo y no creyeron que @ClaudiaLop…
2020-01-28 21:13:17 RT @bgadenuncia: La herencia de @ingrodolfohdez
2020-01-28 21:02:03 RT @CRISTIA68508466: Titular en febrero de @vanguardiacom , extra extra  por fin el viejo pecueco responde x #VITALOGIC 🤣🤣🤣 QUEDO HASTA BON…
2020-01-28 20:46:12 @OtbBga @MauricioMejiaAb @Leonidasgomezg @FerleyProfe @LigaGobernantes Y que creen, que ahora porque están dando qu… https://t.co/OtAn22o4rv
2020-01-28 20:28:52 RT @aalvaradobe: Tenemos que establecer responsabilidades de @AutoviaBP en la avalancha que afectó F/Blanca, menos bosque, menos capacidad…
2020-01-28 20:20:08 @ingrodolfohdez Los partidos tradicionales manipulando las instituciones de poder público @PGN_COL @FiscaliaCol… https://t.co/CkkwfdYfek
2020-01-28 20:10:31 @ingrodolfohdez Ing. Con la voluntad de Dios y del pueblo ud será el futuro presidente de los Colombianos. Lo q 